# Import packages and set parameters

In [ ]:
import numpy as np
from numpy import linalg
from numba import jit
import math
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import SymLogNorm
import matplotlib.colors as colors
import matplotlib.transforms as mtransforms
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)

from scipy.constants import pi
from scipy.interpolate import RectBivariateSpline, griddata

import os
import ast
import pandas as pd

def set_pandas_display_options() -> None:
    """Set pandas display options."""
    # Ref: https://stackoverflow.com/a/52432757/
    display = pd.options.display

    display.max_columns = None
    display.max_rows = None
    display.max_colwidth = None
    display.width = None
    display.chop_threshold = None
    display.precision = 14  # set as needed

set_pandas_display_options()

# Define methods

## Gap detection in DOS

In [ ]:
@jit(nopython=True)
def gap_detector(fermis, IDS, min_IDS_width = 0.001, min_energy_width = 0.1):
    gaps = np.zeros((0,2),dtype=np.float64)
    
    i = 0
    while i <= IDS.size:

        j = 1
        while abs(IDS[i+j]-IDS[i]) < min_IDS_width:
            if j+i == IDS.size-1:
                break
            else:
                j +=1
        else:
            if abs(fermis[i+j]-fermis[i]) > min_energy_width:
                gaps = np.append(gaps,np.array([[fermis[i],fermis[i+j]]]),axis=0)
                i += j
                continue
            else:
                i +=1
                continue
        
        return gaps

## Chern coefficients

In [ ]:
def n_u1t1(theta,Ch_u1t1,Ch_u1u2t1t2):
    return Ch_u1t1 + theta * Ch_u1u2t1t2

def n_u2t2(theta,Ch_u2t2,Ch_u1u2t1t2):
    return Ch_u2t2 + theta * Ch_u1u2t1t2

def n_u1u2(flux,Ch_u1u2,Ch_u1u2t1t2):
    return Ch_u1u2 - flux * Ch_u1u2t1t2

def n_(theta,flux,Ch_,Ch_u1t1,Ch_u2t2,Ch_t1t2,Ch_u1u2t1t2):
    return Ch_ - flux * Ch_t1t2 - theta * Ch_u1t1 - theta * Ch_u2t2 - theta**2 * Ch_u1u2t1t2

# Import data

## Read files for parameters

In [ ]:
files = [x[0] for x in os.walk('./data/')][1::]
df = pd.DataFrame()
for file in files:
    
    data = pd.read_json(file+'/params.json',orient='index').transpose()
    data["key"] = file[7:]
    
    if "n_fermi" in data:
        df = pd.concat([df,data])
    if "n_moments" in data:
        df = pd.concat([df,data])

display(df.set_index('key').sort_index(ascending=False))

## Select data

### skx

In [ ]:
key_fermi, key_morb, key_flux, key_theta = '1747908879', '1747908879', '1709931786', '1710322977'

## Import file and extract data

In [ ]:
# parameter
tex, size        = df.set_index('key').at[key_fermi,'texture'],     df.set_index('key').at[key_fermi,'system_sizes']
theta, flux      = df.set_index('key').at[key_fermi,'q'],           df.set_index('key').at[key_fermi,'flux']
sys_size         = df.set_index('key').at[key_theta,'system_sizes']
t,       m       = df.set_index('key').at[key_theta,'t'],           df.set_index('key').at[key_theta,'m']
shift,   mag     = df.set_index('key').at[key_theta,'shift'],       df.set_index('key').at[key_theta,'mag']

n_energies, n_moments, n_random_states = df.set_index('key').at[key_theta,'n_energies'], df.set_index('key').at[key_theta,'n_moments'], df.set_index('key').at[key_theta,'n_random_states'],
scale, epsilon = 12, 0.01,

n_fermi, min_fermi, max_fermi = df.set_index('key').at[key_fermi,'n_fermi'],  df.set_index('key').at[key_fermi,'min_fermi'], df.set_index('key').at[key_fermi,'max_fermi']

# Data
outdir_fermi = './data/' + key_fermi
outdir_morb  = './data/' + key_morb
outdir_flux  = './data/' + key_flux
outdir_theta = './data/' + key_theta

# DOS
fermis = np.linspace(min_fermi,max_fermi,n_fermi)

spectrum = np.load(outdir_fermi+"/spectrum.npy")

# Chern numbers
# New Convention: tau1 = -t1 and order u1u2t1t2
# u1u2         -> u1u2
# tau1u1       -> (-1)^2 u1t1
# tau2u1       -> (-1)^2 u1t2
# tau1u2       -> (-1)^2 u2t1
# tau2u2       -> (-1)^2 u2t2
# tau1tau2     -> (-1)^2 t1t2
# tau1tau2u1u2 -> (-1)^2 u1u2t1t2

ch_0 = np.array([])
for f in fermis:
    occ = np.array([1 if energy <= f else 0 for energy in spectrum])
    ch_0 = np.append(ch_0,[sum(occ)/size**2])

ch_u1u2     = np.load(outdir_fermi+"/ch_u1u2.npy")
ch_u1t1     = np.load(outdir_fermi+"/ch_tau1u1.npy")
ch_u1t2     = np.load(outdir_fermi+"/ch_tau2u1.npy")
ch_u2t1     = np.load(outdir_fermi+"/ch_tau1u2.npy")
ch_u2t2     = np.load(outdir_fermi+"/ch_tau2u2.npy")
ch_t1t2     = np.load(outdir_fermi+"/ch_tau1tau2.npy")
ch_u1u2t1t2 = np.load(outdir_fermi+"/ch_tau1tau2u1u2.npy")

n_0        = n_(theta,flux,ch_0,ch_u1t1,ch_u2t2,ch_t1t2,ch_u1u2t1t2)     # _
n_u1u2     = n_u1u2(flux,ch_u1u2,ch_u1u2t1t2)                            # u1u2
n_u1t1     = n_u1t1(theta,ch_u1t1,ch_u1u2t1t2)                           # u1t1
n_u1t2     = ch_u1t2                                                     # u1t2
n_u2t1     = ch_u2t1                                                     # u2t1
n_u2t2     = n_u2t2(theta,ch_u2t2,ch_u1u2t1t2)                           # u2t2
n_t1t2     = ch_t1t2                                                     # t1t2
n_u1u2t1t2 = ch_u1u2t1t2                                                 # u1u2t1t2 

# orbital magnetisation
morb = np.load(outdir_morb+"/morb.npy")
morbLC = np.load(outdir_morb+"/morbLC.npy")
morbIC = np.load(outdir_morb+"/morbIC.npy")
    
# data
qs_flux  = np.array([i/sys_size for i in range(1,sys_size)])
qs_theta = np.array([i/sys_size for i in range(1,math.floor(sys_size/2))])

emesh = scale*np.linspace(-1, 1, n_energies) * (1-epsilon)

dos_flux = []
for i in range(len(qs_flux)):
    dos_flux.append( np.load(outdir_flux+'/dos_'+str(i).zfill(4)+'.npy') )
dos_flux = np.array(dos_flux)

dos_theta = []
for i in range(len(qs_theta)):
    dos_theta .append( np.load(outdir_theta+'/dos_'+str(i).zfill(4)+'.npy') )
dos_theta  = np.array(dos_theta )

# double range
qs_theta = np.concatenate((qs_theta,np.array([1-q for q in qs_theta[::-1]])))
dos_theta = np.concatenate((dos_theta,dos_theta[::-1]))

Emin = np.amin(emesh)
Emax = np.amax(emesh)

# Plots

In [ ]:
# Plot parameter
mpl.pyplot.rcdefaults()
plt.rcParams['figure.figsize'] = [20, 10]
plt.rcParams['savefig.facecolor'] = "white"
mpl.rcParams['axes.linewidth'] = 1.2
mpl.rcParams['text.usetex'] = True
mpl.rcParams['mathtext.fontset'] = 'cm'
mpl.rcParams['font.family'] = 'CMU serif'

tfs    = 26 #title font size
lfs    = 20 #label font size
fs     = 18 #font size
cbarfs = 12 #colorbar font size
afs    = 26 #annotation font size
gfs    = 14 #gap font size

# scatter size
s = 1

linthresh = 0.07 # The range within which the plot is linear
linscale = 1   # The factor by which data smaller than `linthresh` is scaled.
norm = SymLogNorm(linthresh=linthresh, linscale=linscale)

## Plot gaps

In [ ]:
gaps = gap_detector(fermis,ch_0,0.001,0.1)

fig, ax = plt.subplots(figsize=(10,10))
ax.plot(fermis,ch_0)
trans = mtransforms.blended_transform_factory(ax.transData, ax.transAxes)

for g in gaps:
    ax.fill_between(fermis, 0, 1, where= np.logical_and(g[0] < fermis, fermis < g[1]) , facecolor='green', alpha=0.5, transform=trans)
    
for g in gaps:
    print(g[0],g[1],np.mean(g))

## Plot Chern numbers

In [ ]:
x_ticks1 = np.arange(0, 3, 1)
x_ticks2 = np.arange(-2, 3, 1)
x_ticks3 = np.arange(-10, 15, 5)
x_ticks4 = np.arange(-10, 15, 5)
x_ticks5 = np.arange(-0.2, 0.3, 0.2)
x_ticks6 = np.arange(-2, 4, 2)
y_ticks = np.arange(-11, 9, 1)
    
fig = plt.figure(figsize=(25,20))
gs = fig.add_gridspec(1,10, hspace=0,wspace=0)
ax1 = plt.subplot(gs.new_subplotspec((0, 0), colspan=1))
ax2 = plt.subplot(gs.new_subplotspec((0, 1), colspan=1))
ax3 = plt.subplot(gs.new_subplotspec((0, 2), colspan=2))
ax4 = plt.subplot(gs.new_subplotspec((0, 4), colspan=2))
ax5 = plt.subplot(gs.new_subplotspec((0, 6), colspan=2))
ax6 = plt.subplot(gs.new_subplotspec((0, 8), colspan=2))

ax1.scatter(n_0,fermis,s=2)
ax1.yaxis.tick_right()
ax1.set_xticks(x_ticks1)
ax1.set_yticks(y_ticks)
ax1.grid(True)
ax1.grid(True,which='minor',color='lightgray')
ax1.set_xlim([3.05,-1.05])
ax1.set_ylabel(r'$E_F \; / \; \lambda_\mathrm{hop}$',fontsize=lfs,labelpad=-10.0)
ax1.yaxis.set_label_position("right")
ax1.set_title(r'$n_\emptyset$',fontsize=24)

ax2.scatter(n_u1u2.real,fermis,s=2)
ax2.yaxis.tick_right()
ax2.set_xticks(x_ticks2)
ax2.set_yticks(y_ticks)
ax2.grid(True)
ax2.grid(True,which='minor',color='lightgray')
ax2.set_xlim([2, -2])
#ax2.set_ylabel(r'$E_F \; / \; \lambda_\mathrm{hop}$',fontsize=lfs,labelpad=-10.0)
ax2.yaxis.set_label_position("right")
ax2.set_title(r'$n_{u_1 u_2}$',fontsize=24)

ax3.scatter(n_t1t2.real,fermis,s=2)
ax3.yaxis.tick_right()
ax3.set_xticks(x_ticks3)
ax3.xaxis.set_minor_locator(MultipleLocator(1))
ax3.set_yticks(y_ticks)
ax3.grid(True)
ax3.grid(True,which='minor',color='lightgray')
ax3.set_xlim([6, -6])
#ax3.set_ylabel(r'$E_F \; / \; \lambda_\mathrm{hop}$',fontsize=lfs,labelpad=-10.0)
ax3.yaxis.set_label_position("right")
ax3.set_title(r'$n_{t_1 t_2}$',fontsize=24)

ax4.scatter(n_u1u2t1t2.real,fermis,s=2)
ax4.yaxis.tick_right()
ax4.set_xticks(x_ticks4)
ax4.xaxis.set_minor_locator(MultipleLocator(1))
ax4.set_yticks(y_ticks)
ax4.grid(True)
ax4.grid(True,which='minor',color='lightgray')
ax4.set_xlim([6, -6])
ax4.set_ylabel(r'$E_F \; / \; \lambda_\mathrm{hop}$',fontsize=lfs,labelpad=-10.0)
ax4.yaxis.set_label_position("right")
ax4.set_title(r'$n_{u_1 u_2 t_1 t_2}$',fontsize=24)

ax5.scatter(morb.real,fermis,s=2,label=r'$M^z_\mathrm{orb}$')
ax5.scatter(morbLC.real-fermis*ch_t1t2.real/(-4*pi),fermis,s=2,c='r',label=r'$M^{LC}_\mathrm{orb}-\mu Ch_{t_1 t_2}$')
ax5.scatter(morbIC.real-fermis*ch_t1t2.real/(-4*pi),fermis,s=2,c='g',label=r'$M^{IC}_\mathrm{orb}-\mu Ch_{t_1 t_2}$')
ax5.yaxis.tick_right()
ax5.set_xticks(x_ticks5)
ax5.xaxis.set_minor_locator(MultipleLocator(0.05))
ax5.set_yticks(y_ticks)
ax5.grid(True)
ax5.grid(True,which='minor',color='lightgray')
ax5.set_xlim([0.4, -0.4])
ax5.set_ylabel(r'$E_F \; / \; \lambda_\mathrm{hop}$',fontsize=lfs,labelpad=-10.0)
ax5.yaxis.set_label_position("right")
ax5.set_title(r'$M^z_\mathrm{orb}\; / \; \frac{e}{\hbar}\lambda_\mathrm{hop}$',fontsize=24)
ax5.legend()

ax6.scatter(morb.real,fermis,s=2,label=r'$M^z_\mathrm{orb}$')
ax6.scatter(morbLC.real,fermis,s=2,c='r',label=r'$M^{LC}_\mathrm{orb}$')
ax6.scatter(morbIC.real,fermis,s=2,c='g',label=r'$M^{IC}_\mathrm{orb}$')
ax6.yaxis.tick_right()
ax6.set_xticks(x_ticks6)
ax6.xaxis.set_minor_locator(MultipleLocator(1))
ax6.set_yticks(y_ticks)
ax6.grid(True)
ax6.grid(True,which='minor',color='lightgray')
ax6.set_xlim([4, -4])
ax6.set_ylabel(r'$E_F \; / \; \lambda_\mathrm{hop}$',fontsize=lfs,labelpad=-10.0)
ax6.yaxis.set_label_position("right")
ax6.set_title(r'$M^z_\mathrm{orb}\; / \; \frac{e}{\hbar}\lambda_\mathrm{hop}$',fontsize=24)
ax6.legend()

for ax in fig.get_axes():
    ax.tick_params(axis='both', which='major', labelsize=20)

## Plot spectrum over flux

In [ ]:
x_ticks = np.arange(0, 1.1, 0.1)

plt.imshow(dos_flux.T, aspect='auto',norm=norm, extent=(0,1, Emin, Emax), interpolation='gaussian', origin = 'lower', resample=True,cmap='Blues');
ax = plt.gca()
ax.set_title(r"$N=$"+str(sys_size)+", "+r"$\lambda_{\mathrm{hop}}$ = "+str(-t)+", "+r"$\lambda_{\mathrm{xc}}$ ="+str(m)+", "+r"$\vartheta$ = "+str(round(theta,4)),fontsize=tfs)
ax.set_xlabel(r"$\phi$",fontsize=lfs)
ax.set_ylabel(r"$E_F$",fontsize=lfs)
ax.set_xticks(x_ticks)
ax.tick_params(axis='both', which='major', labelsize=fs)
ax.set_ylim((-11,11))

cbar = plt.colorbar()
cbar.set_label(label=r'DOS (a.u.)', size=lfs)
cbar.ax.tick_params(labelsize=fs)

## Plot spectrum over theta

In [ ]:
x_ticks = np.arange(0, 1.1, 0.1)

plt.imshow(dos_theta.T, aspect='auto',norm=norm, extent=(0,1, Emin, Emax), interpolation='gaussian', origin = 'lower', resample=True,cmap='Blues');
ax = plt.gca()
ax.set_title(r"$N=$"+str(sys_size)+", "+r"$\lambda_{\mathrm{hop}}$ = "+str(-t)+", "+r"$\lambda_{\mathrm{xc}}$ ="+str(m)+", "+r"$\varphi$ = "+str(round(flux,4)),fontsize=tfs)
ax.set_xlabel(r"$\theta$",fontsize=lfs)
ax.set_ylabel(r"$E_F$",fontsize=lfs)
ax.set_xticks(x_ticks)
ax.tick_params(axis='both', which='major', labelsize=fs)
ax.set_ylim( (-11,8))

cbar = plt.colorbar()
cbar.set_label(label=r'DOS (a.u.)', size=lfs)
cbar.ax.tick_params(labelsize=fs)

# DoS + Chern

In [ ]:
gaps = gap_detector(fermis,ch_0,0.001,0.1)
print(gaps.shape)

In [ ]:
# Plot parameter
mpl.pyplot.rcdefaults()
plt.rcParams['figure.figsize'] = [20, 10]
plt.rcParams['savefig.facecolor'] = "white"
mpl.rcParams['axes.linewidth'] = 1.2
mpl.rcParams['text.usetex'] = True
mpl.rcParams['mathtext.fontset'] = 'cm'
mpl.rcParams['font.family'] = 'CMU serif'

tfs    = 24 #title font size
lfs    = 24 #label font size 
fs     = 22 #font size
cbarfs = 18 #colorbar font size
afs    = 24 #annotation font size
gfs    = 24 #gap font size

# scatter size
s = 2

linthresh = 0.07 # The range within which the plot is linear
linscale = 1   # The factor by which data smaller than `linthresh` is scaled.
norm = SymLogNorm(linthresh=linthresh, linscale=linscale)

x_ticks0 = np.arange(0.1, 0.5, 0.1)  
x_ticks1 = np.arange(0, 3, 1)
x_ticks2 = np.arange(-0, 2, 1)
x_ticks3 = np.arange(-2, 8, 2)
x_ticks4 = np.arange(-2, 4, 2)
x_ticks5 = np.arange(-1, 1, 0.2)
x_ticks6 = np.arange(0.1, 0.5, 0.1)  
y_ticks = np.arange(-10, 10, 2.5)
    
fig = plt.figure()
gs = fig.add_gridspec(1,40, hspace=0,wspace=0)
ax0 = plt.subplot(gs.new_subplotspec((0, 0), colspan=8))
ax1 = plt.subplot(gs.new_subplotspec((0, 8), colspan=3),sharey=ax0)
ax2 = plt.subplot(gs.new_subplotspec((0, 11), colspan=3),sharey=ax0)
ax3 = plt.subplot(gs.new_subplotspec((0, 14), colspan=6),sharey=ax0)
ax4 = plt.subplot(gs.new_subplotspec((0, 20), colspan=6),sharey=ax0)
ax5 = plt.subplot(gs.new_subplotspec((0, 26), colspan=6),sharey=ax0)
ax6 = plt.subplot(gs.new_subplotspec((0, 32), colspan=8),sharey=ax0)

# fig.suptitle(tex+": "+r"$N=$"+str(sys_size)+", "+r"$\lambda_{\mathrm{hop}}$ = "+str(-t)+", "+r"$\lambda_{\mathrm{xc}}$ ="+str(m)+", "+r"$\vartheta$ = "+str(round(theta,4))+", "+r"$\varphi$ = "+str(round(flux,4))+", "+r"shift = "+str(round(shift/np.pi,1))+"$\pi$"+", "+r"$m$ = "+str(mag),fontsize=tfs)

ax0.imshow(dos_theta.T, aspect='auto',norm=norm, extent=(0,1, Emin, Emax), interpolation='gaussian', origin = 'lower', resample=True,cmap='Blues');
ax0.set_xticks(x_ticks0)
ax0.set_xlim([0,0.5])
ax0.xaxis.set_minor_locator(MultipleLocator(0.05))
ax0.set_title(r"$\mathrm{DOS}$",fontsize=tfs)
ax0.set_ylabel(r'$E_F \; / \; \lambda_\mathrm{hop}$',fontsize=lfs,labelpad=-10.0)
ax0.set_xlabel(r"$\vartheta$",fontsize=lfs)

ax0.axvline(x = theta, color = 'red', label = r'$\theta$')
ax0.annotate(r'$\vartheta=\frac{{ {:.0f} }}{{ {:.0f} }}$'.format(theta*size,size),(theta-0.15,-0.6),color='red',fontsize=afs)

ax0.annotate(r'$U_1$',(0.43,2.8),color='black',fontsize=gfs)
ax0.annotate(r'$U_2$',(0.43,4.8),color='black',fontsize=gfs)
ax0.annotate(r'$U_3$',(0.43,6.3),color='black',fontsize=gfs)

ax0.annotate(r'$C_0$',(0.43,-0.8),color='black',fontsize=gfs)

ax0.annotate(r'$L_1$',(0.43,-8.5),color='black',fontsize=gfs)
ax0.annotate(r'$L_2$',(0.43,-6.7),color='black',fontsize=gfs)
ax0.annotate(r'$L_3$',(0.43,-5.3),color='black',fontsize=gfs)

ax1.scatter(n_0,fermis,s=2)
ax1.set_xticks(x_ticks1)
ax1.xaxis.set_minor_locator(MultipleLocator(1))
ax1.grid(True,which='minor',color='lightgray',axis='x')
ax1.set_xlim([-0.5,2.5])
ax1.set_title(r'$n_\emptyset$',fontsize=tfs,y=1.005)

ax2.scatter(n_u1u2.real,fermis,s=2)
ax2.set_xticks(x_ticks2)
ax2.xaxis.set_minor_locator(MultipleLocator(1))
ax2.grid(True,which='minor',color='lightgray',axis='x')
ax2.set_xlim([-0.5, 1.5])
ax2.set_title(r'$n_{u_1 u_2}$',fontsize=tfs,y=1.005)

ax3.scatter(n_t1t2.real,fermis,s=2)
ax3.set_xticks(x_ticks3)
ax3.xaxis.set_minor_locator(MultipleLocator(1))
ax3.grid(True,which='minor',color='lightgray',axis='x')
ax3.set_xlim([-4, 8])
ax3.set_title(r'$n_{t_1 t_2}$',fontsize=tfs,y=1.005)

ax4.scatter(n_u1u2t1t2.real,fermis,s=2)
ax4.set_xticks(x_ticks4)
ax4.xaxis.set_minor_locator(MultipleLocator(1))
ax4.grid(True,which='minor',color='lightgray',axis='x')
ax4.set_xlim([-4, 4])
ax4.set_title(r'$n_{u_1 u_2 t_1 t_2}$',fontsize=tfs,y=1.005)

ax5.scatter(morb.real,fermis,s=2,label=r'$M^z_\mathrm{orb}$')
ax5.scatter(morbLC.real-fermis*ch_t1t2.real/(-4*pi),fermis,s=2,c='r',label=r'$M^{LC}_\mathrm{orb}+\frac{E_F}{4\pi} Ch_{t_1 t_2}$')
ax5.scatter(morbIC.real-fermis*ch_t1t2.real/(-4*pi),fermis,s=2,c='g',label=r'$M^{IC}_\mathrm{orb}+\frac{E_F}{4\pi} Ch_{t_1 t_2}$')
ax5.set_xticks(x_ticks5)
ax5.xaxis.set_minor_locator(MultipleLocator(0.05))
ax5.grid(True,which='minor',color='lightgray',axis='x')
ax5.set_xlim([-0.3, 0.3])
ax5.set_title(r'$M^z_\mathrm{orb}\; / \; \frac{e}{\hbar}\lambda_\mathrm{hop}$',fontsize=tfs,y=1.01)
ax5.legend(loc = (0.10,0.48), fontsize= 14, markerscale=4.0)

ax6.imshow(dos_flux.T, aspect='auto',norm=norm, extent=(0,1, Emin, Emax), interpolation='gaussian', origin = 'lower', resample=True,cmap='Blues');
ax6.set_xticks(x_ticks6)
ax6.set_xlim([0,0.5])
ax6.xaxis.set_minor_locator(MultipleLocator(0.05))
ax6.set_title(r"$\mathrm{DOS}$",fontsize=tfs,y=1.005)
ax6.set_xlabel(r"$\phi$",fontsize=lfs)

ax6.axvline(x = flux, color = 'red', label = r'$flux$')
ax6.annotate(r'$\phi=\frac{{ {:.0f} }}{{ {:.0f} }}$'.format(flux*size,size),(flux+0.01,-0.6),color='red',fontsize=afs)

ax6.annotate(r'$U_1$',(0.01,2.8),color='black',fontsize=gfs)
ax6.annotate(r'$U_2$',(0.01,4.8),color='black',fontsize=gfs)
ax6.annotate(r'$U_3$',(0.01,6.3),color='black',fontsize=gfs)

ax6.annotate(r'$C_0$',(0.01,-0.8),color='black',fontsize=gfs)

ax6.annotate(r'$L_1$',(0.01,-8.5),color='black',fontsize=gfs)
ax6.annotate(r'$L_2$',(0.01,-6.7),color='black',fontsize=gfs)
ax6.annotate(r'$L_3$',(0.01,-5.3),color='black',fontsize=gfs)

for ax in fig.get_axes():
    ax.yaxis.set_minor_locator(MultipleLocator(0.5))
    ax.tick_params(axis='both', which='major', labelsize=fs)
    ax.set_yticks(y_ticks)
    ax.set_ylim([-11,8])

for ax in fig.get_axes()[1:-1]:
    ax.grid(True,color='gray',axis='x')
        
gaps = gap_detector(fermis,ch_0,0.001,0.1)
for g in gaps:
    ax0.fill_between(np.arange(theta,0.51,0.001),g[0],g[1], facecolor='gray', alpha=0.5)
    ax6.fill_between(np.arange(0.0,flux,0.001),g[0],g[1], facecolor='gray', alpha=0.5)
    for ax in fig.get_axes()[1:-1]:
        ax.fill_between(np.arange(-30,30,0.01),g[0],g[1], facecolor='gray', alpha=0.5)
        
for ax in fig.get_axes()[1:]:
    ax.yaxis.tick_right()

plt.savefig("./Plots/skx_flux_Chernhierarchy_kpm_morb.png", dpi=150, bbox_inches='tight')